# Python Excel tools performance comparison #

There are two tests:
- Test 1: Values - A new workbook with worksheet cell values from testData[ROWS][COLS].
- Test 2: Formatting - A new workbook with worksheet cell formatting from formatData[ROWS][COLS].

The Jupyter Notebook magic command %%timeit runs each test 7 times and give a performance summary below the code cell.

Python Excel tools tested:
- SpreadsheetGear with pythonnet
- PyExcelerate
- XLSXWriter
- OpenPyXL

# Install packages #

In [ ]:
%pip install pyexcelerate
%pip install openpyxl
%pip install xlsxwriter
%pip install pythonnet

# Initialize test data and output folder

In [2]:
import os
import random

ROWS = 1000
COLUMNS = 100
BOLD = 1
ITALIC = 2
UNDERLINE = 4
RED_BG = 8

# global testData, formatData
testData = []
formatData = []
for row in range(ROWS):
    testDataRowCells = []
    formatDataRowCells = []
    for col in range(COLUMNS):
        testDataRowCells.append(random.randint(1,100))
        formatDataRowCells.append(random.randint(1, (1 << 4) - 1))
    testData.append(testDataRowCells)
    formatData.append(formatDataRowCells)

# initialize output folder
outputFolder =  os.path.join(os.getcwd(), "output")
if not os.path.exists(outputFolder):
    os.makedirs(outputFolder)

# SpreadsheetGear with pythonnet tests #

Python doesn't support setting a Nuget package reference which is how the .NET6 SpreadsheetGear library is distributed. 

My workaround: 
1. Download the [SpreadsheetGear v9 Nuget package](https://www.nuget.org/packages/SpreadsheetGear/#versions-body-tab)
2. Rename its extension to .zip and unzip it 
3. Copy its .NET6 SpreadsheetGear.dll to the project workspace folder. In this example this is the Samples_JupyterPython folder.

In [3]:
# Initialize SpreadsheetGear and pythonnet
import os
import pythonnet
if (pythonnet._RUNTIME==None):
    # Change the runtime_config path to your .NET SDK 6.0 dotnet.runtimeconfig.json file
    pythonnet.load("coreclr",runtime_config="C:\\Program Files\\dotnet\\sdk\\6.0.404\\dotnet.runtimeconfig.json")
    # import the pythonnet Common Language Runtime module
    import clr
    # import the pythonnet System namespace
    import System

    SpreadsheetGearAssemblyPath = os.path.join(os.getcwd(), "SpreadsheetGear.dll")
    clr.AddReference(SpreadsheetGearAssemblyPath)
    print("Added assembly reference: " + SpreadsheetGearAssemblyPath)
    import SpreadsheetGear

    # SpreadsheetGear License
    # SpreadsheetGear for .NET Standard:  Free use is limited to 1,000 rows x 100 columns x 10 worksheets x 3 workbooks. 
    # Visit https://www.spreadsheetgear.com/downloads/signedlicense.aspx for more information.
    # SpreadsheetGear.Factory.SetSignedLicense("UNCOMMENT THIS LINE AND PUT YOUR SIGNED LICENSE HERE")

# Note: If you run into the SpreadsheetGear FREE use limition you can "Restart Kernel" for this Jupyter Notebook to reset it.

Added assembly reference: c:\Users\Chris Hamilton\source\repos\SpreadsheetGearCodeSamples_VSCode\Samples_JupyterPython\SpreadsheetGear.dll


### Test 1: Values - SpreadsheetGear with pythonnet ###

In [4]:
%%timeit
# Note: Remove or comment out %%timeit if you run into the SpreadsheetGear FREE use  
# limitation of 1,000 rows x 100 columns x 10 worksheets x 3 workbooks.

# Build .NET 2-dimensional object array that SpreadsheetGear uses for fast loading of values
objectValues = System.Array.CreateInstance(System.Object, ROWS, COLUMNS)
for row in range(ROWS):
    for col in range(COLUMNS):
        objectValues[row, col] = float(testData[row][col])

workbook = SpreadsheetGear.Factory.GetWorkbook()
worksheet = workbook.Worksheets[0]
worksheet.Cells[0,0,ROWS-1,COLUMNS-1].Value = objectValues
workbook.SaveAs(os.path.join(outputFolder,"SpreadsheetGearValues.xlsx"), SpreadsheetGear.FileFormat.OpenXMLWorkbook)

58.8 ms ± 9.42 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Test 2: Formatting - SpreadsheetGear with pythonnet ###

In [5]:
%%timeit
# Note: Remove or comment out %%timeit if you run into the SpreadsheetGear FREE use  
# limitation of 1,000 rows x 100 columns x 10 worksheets x 3 workbooks.

workbook = SpreadsheetGear.Factory.GetWorkbook()
worksheet = workbook.Worksheets[0]
cells = worksheet.Cells
for row in range(ROWS):
    for col in range(COLUMNS):
            cell = worksheet.Cells[row, col, row, col]
            #cell.Value = float(formatData[row][col])
            if ((formatData[row][col] & ITALIC) == ITALIC):
                cell.Font.Italic = True
            if ((formatData[row][col] & BOLD) == BOLD):
                cell.Font.Bold = True
            if ((formatData[row][col] & UNDERLINE) == UNDERLINE):
                cell.Font.Underline = SpreadsheetGear.UnderlineStyle.Single
            if ((formatData[row][col] & RED_BG) == RED_BG):
                cell.Interior.Color = SpreadsheetGear.Colors.Red

workbook.SaveAs(os.path.join(outputFolder,"SpreadsheetGearFormats.xlsx"), SpreadsheetGear.FileFormat.OpenXMLWorkbook)

878 ms ± 7.87 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# PyExcelerate tests #

In [6]:
import pyexcelerate

### Test 1: Values - PyExcelerate ###

In [7]:
%%timeit
# Populate worksheet cells with values from testData[ROWS][COLS]
wb = pyexcelerate.Workbook()
wb.new_sheet("testData", data=testData)
wb.save(os.path.join(outputFolder,"PyExcelerateValues.xlsx"))

225 ms ± 1.71 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Test 2: Formatting - PyExcelerate ###

In [8]:
%%timeit
# Format worksheet cells with formatting from formatData[ROWS][COLS]
wb = pyexcelerate.Workbook()
ws = wb.new_sheet("formatting")
bold = pyexcelerate.Style(font=pyexcelerate.Font(bold=True))
italic = pyexcelerate.Style(font=pyexcelerate.Font(italic=True))
underline = pyexcelerate.Style(font=pyexcelerate.Font(underline=True))
red = pyexcelerate.Style(fill=pyexcelerate.Fill(background=pyexcelerate.Color(255, 0, 0, 0)))
for row in range(ROWS):
    for col in range(COLUMNS):
        ws.set_cell_value(row + 1, col + 1, 1)
        style = pyexcelerate.Style()
        if formatData[row][col] & BOLD:
            style.font.bold = True
        if formatData[row][col] & ITALIC:
            style.font.italic = True
        if formatData[row][col] & UNDERLINE:
            style.font.underline = True
        if formatData[row][col] & RED_BG:
            style.fill.background = pyexcelerate.Color(255, 0, 0)
        ws.set_cell_style(row + 1, col + 1, style)
wb.save(os.path.join(outputFolder,"PyExcelerateFormats.xlsx"))


598 ms ± 2.74 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# XLSXWriter tests

In [9]:
import xlsxwriter.workbook

### Test 1: Values - XLSXWriter ###

In [10]:
%%timeit
# Populate worksheet cells with values from testData[ROWS][COLS]
wb = xlsxwriter.workbook.Workbook(os.path.join(outputFolder,"XLSXWriterValues.xlsx"), {"constant_memory": True})
ws = wb.add_worksheet()
for row in range(ROWS):
    for col in range(COLUMNS):
        ws.write_number(row, col, 1)
wb.close()

217 ms ± 1.69 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Test 2: Formatting - XLSXWriter ###

In [11]:
%%timeit
# Format worksheet cells with formatting from formatData[ROWS][COLS]
wb = xlsxwriter.workbook.Workbook(os.path.join(outputFolder,"XLSXWriterFormats.xlsx"), {"constant_memory": True})
ws = wb.add_worksheet()
cell_formats = []
for i in range(16):
    cell_format = wb.add_format()
    if i & BOLD:
        cell_format.set_bold()
    if i & ITALIC:
        cell_format.set_italic()
    if i & UNDERLINE:
        cell_format.set_underline()
    if i & RED_BG:
        cell_format.set_bg_color("red")
    cell_formats.append(cell_format)

for row in range(ROWS):
    for col in range(COLUMNS):
        ws.write_number(row, col, 1, cell_formats[formatData[row][col]])
wb.close()

316 ms ± 7.07 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# OpenPyXL tests

In [12]:
import openpyxl

### Test 1: Values - OpenPyXL ###

In [13]:
%%timeit
# Populate worksheet cells with values from testData[ROWS][COLS]
wb = openpyxl.workbook.Workbook(write_only=True)
ws = wb.create_sheet()
ws.title = "Test 1"
for row in testData:
    ws.append(row)
wb.save(os.path.join(outputFolder,"OpenPyXLValues.xlsx"))

439 ms ± 3.79 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Test 2: Formatting - OpenPyXL ###

In [14]:
%%timeit
# Format worksheet cells with formatting from formatData[ROWS][COLS]
wb = openpyxl.workbook.Workbook()
ws = wb.create_sheet()
ws.title = "Test 1"
for col_idx in range(COLUMNS):
    col = openpyxl.utils.get_column_letter(col_idx + 1)
    for row in range(ROWS):
        ws["%s%s" % (col, row + 1)].value = 1
        if formatData[row][col_idx] & BOLD:
            ws["%s%s" % (col, row + 1)].font = openpyxl.styles.Font(bold=True)
        if formatData[row][col_idx] & ITALIC:
            ws["%s%s" % (col, row + 1)].font = openpyxl.styles.Font(italic=True)
        if formatData[row][col_idx] & UNDERLINE:
            ws["%s%s" % (col, row + 1)].font = openpyxl.styles.Font(
                underline="single"
            )
        if formatData[row][col_idx] & RED_BG:
            ws["%s%s" % (col, row + 1)].fill = openpyxl.styles.PatternFill(
                fill_type=openpyxl.styles.fills.FILL_SOLID,
                start_color=openpyxl.styles.Color(openpyxl.styles.colors.BLUE),
                end_color=openpyxl.styles.Color(openpyxl.styles.colors.BLUE),
            )
        ws["%s%s" % (col, row + 1)].value = 1
wb.save(os.path.join(outputFolder,"OpenPyXLFormats.xlsx"))

4.35 s ± 13.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
